In [ ]:
import os 
import re
import math
import numpy as np
import ast
from functools import cmp_to_key
from collections import defaultdict, deque

PATH = '/Users/xuyaozhang/Desktop/AdventofCode2022/input'


def read_txt(filename):
    with open(filename) as data_file:
            
        lines = data_file.read().split('\n') 
        return lines
            
def rotate(s, o):
    res = s
    if o == '<':
        if s[0] != '#':
            res = s[1:] + s[:1]
        
    else:
        if s[-1] != '#':
            res = s[-1] + s[:-1]
    return res

def check_border(rock, o):
    check = False
    for r in rock:
        if o == '<':
            if r[0] == '#':
                return True
        else:
            if r[-1] == '#':
                return True
    return check

def check_block(rock, o):
    check = False
    for r in rock:
        if o == '<':
            if r[0] == '#':
                return True
            for i in range(1, 7):
                if r[i] == '#' and r[i-1] == 'x':
                    return True
                
        else:
            if r[-1] == '#':
                return True
            for i in range(0, 6):
                if r[i] == '#' and r[i+1] == 'x':
                    return True
    return check

def rotate_rocks(rock, o):
    """
    This function rotates a rock before it reaches the fixed rocks
    """
    if not check_border(rock, o):
        return [rotate(r, o) for r in rock]
    else:
        return rock
def combine_c(c1, c2):
    if c1 == '.' and c2 == '.':
        return '.'
    elif c1 == '.' and c2 == '#':
        return c2
    elif c1 == 'x' and c2 == '.':
        return c1
    elif c1 == 'x' and c2 == 'x':
        return c1
    elif c1 == '.' and c2 == 'x':
        return c1
          

def combine_strs(str1, str2):
    res = ''
    for i in range(7):
        res += combine_c(str1[i], str2[i])
    return res

def check_godown(combine, h_x, l_x):
    """
    This function checks if the rock can go down, we only check the lines where this block appear (h_x to l_x+1)
    """
    for i in range(h_x, l_x + 1):
        for p, c in enumerate(combine[i]):
            if c == '#' and combine[i+1][p] == 'x':
                return False
    return True

def godown(combine, h_x, l_x):
    """
    This function move the rock one unit down
    Return the result after going down, the high and low index of current rock 
    """
    start = l_x+1
    while start >0:
        combine[start] = combine_strs(combine[start], combine[start-1])
        combine[start-1] = combine[start-1].replace("#", ".")
        start -= 1
    if combine[0] == '.......':
        combine.pop(0)
    else:
        h_x += 1
        l_x += 1
        
    return combine, h_x, l_x

def checkleftright(combine, h_x, l_x, op):
    """
    This function checks if the rock can move left or right, we only check the lines where this block appear (h_x to l_x+1)
    """
    sub = combine[h_x:l_x+1]
     
    return check_block(sub, op)


def move_rocks(combine, h_x, l_x, op):
    """
    This function rotates a rock when it intersects the fixed rocks
    """
    for i in range(h_x, l_x+1):
        l = list(combine[i])
        if op == '<':
            for j in range(1,7):
                if l[j] == '#':
                    l[j-1] = '#'
                    l[j] = '.'
        else:
            l.reverse()
            for j in range(1,7):
                if l[j] == '#':
                    l[j-1] = '#'
                    l[j] = '.'
            l.reverse()
        combine[i] = ''.join(l)
        
    return combine, h_x, l_x
    
        
def fall_to_rest(fixed, current, curheight, op_idx, ops, l_op):
    """
    fixed: fixed area
    current: current rock
    curheight: the height of the current rock
    op_idx: operator index
    ops: list of operators
    l_op: lengh of operators
    """
    
    current = rotate_rocks(current, ops[op_idx%l_op])
    
    combine = current + fixed
    
    h_x = 0
    l_x = curheight - 1
    rest = False
    op_idx += 1
    while not rest:
        if not check_godown(combine, h_x, l_x):
            rest = True
            for i in range(h_x, l_x + 1):
                combine[i] = combine[i].replace("#", "x")
            
        else:
            combine, h_x, l_x = godown(combine, h_x, l_x)
            
            if not checkleftright(combine, h_x, l_x, ops[op_idx%l_op]):
                combine, h_x, l_x = move_rocks(combine, h_x, l_x, ops[op_idx%l_op])
            op_idx += 1
            
                
    return combine, op_idx
                

def solution1(nums, rocks, operators, l_op):
    area = ['xxxxxxx'] #start with floor
    plus = 3
    i = 0
    op_idx = 0
    while i<nums:
        rock = rocks[i%5]
        for t in range(plus):
            rock = rotate_rocks(rock, operators[op_idx%l_op])
            op_idx += 1
    
        area, op_idx = fall_to_rest(area, rock, len(rock), op_idx, operators, l_op)
        i += 1


    res = len(area) - 1
    return res


def solution2(nums, rocks, operators, l_op):
    """
    It is not possible to use the same method in part 1 and only change the numbers. There must be a pattern that the rocks
    repeat previous behavior and shape the same image.
    
    """
    area = ['xxxxxxx'] #start with floor
    plus = 3
    i = 0
    op_idx = 0
    visited = {}
    heights = []
    height = 0
    while i<nums:
        
        
        rock = rocks[i%5]
        for t in range(plus):
            rock = rotate_rocks(rock, operators[op_idx%l_op])
            op_idx += 1
    
        area, op_idx = fall_to_rest(area, rock, len(rock), op_idx, operators, l_op)
        height = len(area)-1
        heights.append(height)
        visited[(tuple(area[:-1]), op_idx%l_op, i%5)] = i
        
        if len(area)%2 != 0:
            middle = (len(area) -1)//2
            if (tuple(area[middle:-1]), op_idx%l_op, i%5) in visited:
                j = visited[(tuple(area[middle:-1]), op_idx%l_op, i%5)] 
                if j < i:
                    return heights[j + (nums - j) % (i - j)] + (nums - j) // (i - j) * (height - heights[j]) - 1
        
        
        i += 1


    
    return height

    

In [ ]:
operators = read_txt(PATH+'/day17.txt')[0]
l_op = len(operators)
rocks = [['..####.'], 
         
         ['...#...',
          '..###..',
          '...#...'],
         
         ['....#..',
          '....#..',
          '..###..'],
         
         ['..#....',
          '..#....',
          '..#....',
          '..#....'],
         
         ['..##...',
          '..##...']
        ]

print(solution1(2022, rocks, operators, l_op)) # ans1
print(solution2(1000000000000, rocks, operators, l_op)) # ans2